# Caso 3: Good Reads 

## Sección Tercera: Análisis Predictivo

### Objetivos:

1. Una editorial nos ha contactado para ver qué parámetros debería tener un libro para que fuera exitoso. A partir del dataset y su análisis, orienta a la editorial sobre qué parámetros deben seguir a la hora de publicar un nuevo libro.

2. Diseña un modelo que, a partir de un libro de entrada, te recomiende una nueva lectura. Puedes utilizar o bien el dataset proporcionado o bien enriquecerlo (por ejemplo, utilizando técnicas de __webscrapping__, o añadiendo más atributos a los libros actuales).
 
3. Respecto a este sistema, a modo de ejemplo, explica las recomendaciones que proporcionaría el modelo si entráramos los siguientes libros: 
* "**A Court of Thorns and Roses**" de __Sarah J. Maas__
* "**Hamlet**" de __William Shakespeare__
* "**La Apología de Sócrates**" de __Platón__

### Metodología

1. Procesamiento de herramientas de webscrapping para obtener la data del sitio "**Good Reads**". Estas herramientas son los archivos `get_books.py` y `get_books_from_list.py`, que fueron suministrados con el ejercicio.
2. Agregar la data obtenida al dataset de good reads provisto por el ejercicio.
3. Procesar los archivos de metadata que están en formato __JSON__ para construir los dataframes necesarios para hacer el modelo de predicción.
4. Construir la matriz de similitud utilizando cosine_similarity
5. Crear la función de predicción

___

### Instalar dependencias

In [ ]:
!pip install -r ./../requirements.txt

### Obtenemos los archivos de metadata para mayo y junio 2024

In [ ]:
!python get_books_from_list.py --url_path https://www.goodreads.com/list/show/201106.Best_books_of_May_2024 --output_directory_path ./dataset/my_list_of_books_may.txt

In [ ]:
!python get_books_from_list.py --url_path https://www.goodreads.com/list/show/202186.Best_books_of_June_2024 --output_directory_path ./dataset/my_list_of_books_jun.txt

In [ ]:
!python get_books.py --book_ids_path ./dataset/my_list_of_books_may.txt --output_directory_path ./data/classic_book_metadata

In [ ]:
!python get_books.py --book_ids_path ./dataset/my_list_of_books_jun.txt --output_directory_path ./data/classic_book_metadata

### Descomprimimos el dataset LSGoodReads 

In [ ]:
from caso03.descomprimir_dataset import unzip_dataset
# Carga de datos
# Cargar dataset y descomprimir en /datos
unzip_dataset("./dataset/LSGoodReads.zip","./data")

### Procesamos los archivos de metadata encontrados en __"./data/classic_book_metadata"__ y en __"./data/LSGoodReads"__

Definimos los imports y las funciones de uso general

In [ ]:
import os
import re
import pandas as pd

def to_float(cell_value) -> float:
    if type(cell_value) == str and cell_value.isnumeric():
        return float(cell_value)
    elif type(cell_value) == str and cell_value.find("k") > -1:
        return float(cell_value.replace('k', ''))*1000
    elif type(cell_value) == int:
        return float(cell_value)
    return 0.0

def clean_text(cell_value) -> str:
    text = str(cell_value)                      # Convert the input text to string
    text = text.strip()
    text = text.lower()                         # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)            # replace repeated blanks with a single one
    text = re.sub(r' ', '-', text)              # replace blanks with '-'
    text = re.sub(r'[^a-zA-Z0-9\-]', '', text)  # Remove special characters except for digits, letters and '-'
    return text

def list_to_str(a_list: list) -> str:
    if type(a_list) is list:
        another_list = sorted(a_list)
        another_list = [ clean_text(cell_value) for cell_value in list(set(another_list)) ]
        return " ".join(another_list)
    return ""

def get_books_metadata(path_to_json: str) -> pd.DataFrame:
    df = pd.read_json(path_to_json)
    df.set_index('book_id', inplace=True)
    
    # Guardamos el título original
    df['original_title'] = df['book_title'].apply(lambda title: str(title).strip())
    
    # Limpiamos textos
    for column_name in ['book_title', 'author', 'book_language', 'format']:
        df[column_name] = df[column_name].map(clean_text)            
    
    # Limpiar numéricos
    for column_name in ['num_pages', 'num_ratings', 'num_reviews', 'year_first_published', 'people_curr_read', 'peop_want_to_read']:
        df[column_name] = df[column_name].map(to_float)

    # Limpiar listas
    for column_name in ['book_series', 'book_settings', 'book_characters', 'genres', 'awards']:
        df[column_name] = df[column_name].map(list_to_str)            

    # Limpiar ratings in rating_distribution
    df['positive_ratings'] = 0.0
    df['negative_ratings'] = 0.0
    for index, row in df.iterrows():
        reviews = row['rating_distribution']
        positive_ratings = 0.0
        negative_ratings = 0.0
        if reviews:
            positive_ratings = float(reviews['5 Stars']) + float(reviews['4 Stars']) + float(reviews['3 Stars'])
            negative_ratings = float(reviews['2 Stars']) + float(reviews['1 Star'])
        df.loc[index, 'positive_ratings'] = positive_ratings
        df.loc[index, 'negative_ratings'] = negative_ratings

    # remove not needed columns
    df.drop(['book_id_title', 'cover_image_uri', 'authorlink', 'rating_distribution'], inplace=True, axis=1)
    return df

**Exploramos los datos cargados de "classic_book_metadata"**

In [ ]:
classics_all_books_path = os.path.abspath(
    os.path.join('./data/classic_book_metadata', 'all_books.json')
)
books_df = pd.DataFrame(get_books_metadata(classics_all_books_path))
print(books_df.shape)
print(books_df.dtypes)
books_df.head(50)

**Exploramos los libros de "LSGoodReads"**

In [ ]:
goodreads_all_books_path = os.path.abspath(
    os.path.join('./data/LSGoodReads', 'all_books.json')
)
books_df = pd.concat([books_df, pd.DataFrame(get_books_metadata(goodreads_all_books_path))]) 
print(books_df.shape)
print(books_df.dtypes)
books_df.head(100)

#### Conclusiones de la exploración inicial
Después de esta corta exploración, observamos que la data es de buena calidad, y es posible realizar los siguientes procesos.

___


### Exploración de datos.

In [ ]:
# Revisamos la distribución de las variables numéricas   
books_df.hist(figsize = (10, 10))

In [ ]:
# Nos hacemos una idea general de las variables categóricas
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def generate_word_cloud(text_column, column_name):
    wordcloud = WordCloud(background_color='white').generate(' '.join(text_column.unique()))
    plt.figure(figsize=(7,5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(column_name)
    plt.show()

for column_name in books_df.select_dtypes(include='object').columns.tolist():
    generate_word_cloud(books_df[column_name], column_name)

___

Preparamos la data para crear el tensor requerido para calcular la matriz de similitud del coseno

In [ ]:
# Rellenamos los vacíos.
for column_name in books_df.select_dtypes(include='number').columns:
    books_df[column_name] = books_df[column_name].fillna(0.0)

# Calculamos la bolsa de tokens para la matriz 
books_df['book_tokens'] = ""
for index, row in books_df.iterrows():
    data = [row[column_name] for column_name in books_df.select_dtypes(include=['object']).columns]
    data = map(lambda tokens: re.sub(r'[\-]', ' ', tokens), data) 
    cell = " ".join([item for item in data if item != ""])
    books_df.loc[index, 'book_tokens'] = cell.strip()

books_df[['book_title','book_tokens']].head(10)


In [ ]:
# Creamos las funciones para obtener la metadata a partir del título
def get_book_id_by_title(df: pd.DataFrame, book_title:str) -> int|None:
    # probar primero contra título original:
    book_index = df.index[df['original_title'] == book_title].tolist()
    if book_index:
        return book_index[0]    
    # Luego contra el título preprocesado    
    title_transformed = clean_text(book_title)
    book_index = df.index[df['book_title'] == title_transformed].tolist()
    if book_index:
        return book_index[0]
    # No se encontró
    return None

def get_book_original_title_by_id(df: pd.DataFrame, book_id:int) -> str|None:
    if not df.loc[book_id].empty:
        return df.loc[book_id]['original_title']
    return None

def get_book_title_by_id(df: pd.DataFrame, book_id:int) -> str|None:
    if not df.loc[book_id].empty:
        return df.loc[book_id]['book_title']
    return None

In [ ]:
# Probamos las funciones creadas con:
#   "A Court of Thorns and Roses" de Sarah J. Maas
#   "Hamlet" de William Shakespeare
#   "La Apología de Sócrates" de Platón

print(clean_text("A Court of Thorns and Roses"))
print(clean_text("Hamlet"))
print(clean_text("La Apologia de Socrates"))
print(clean_text("La Apología de Sócrates"))

print(get_book_id_by_title(books_df, "A Court of Thorns and Roses"))
print(get_book_id_by_title(books_df, "La Apologia de Socrates"))
print(get_book_id_by_title(books_df, "La Apología de Sócrates"))
print(get_book_id_by_title(books_df, "Hamlet"))
print(get_book_original_title_by_id(books_df, 1420))
books_df.query(f"original_title=='Hamlet'")

In [ ]:
# Calculamos el tensor de la variable categórica: 'book_tokens' como matriz dispersa:
from sklearn.feature_extraction.text import TfidfVectorizer

categorical_attrs_df = books_df['book_tokens'].copy()
tfidf_vectorizer=TfidfVectorizer()
tfidf_vectors=tfidf_vectorizer.fit_transform(categorical_attrs_df)

In [ ]:
# Características de la variable categórica 'book_tokens' por book_id.
tensor_categorical_df = pd.DataFrame(
    tfidf_vectors.todense(),
    index=categorical_attrs_df.index,
    columns=tfidf_vectorizer.get_feature_names_out()
)
tensor_categorical_df.head()

In [ ]:
# Obtenemos una copia de las variables numéricas para calcular el tensor.
numerical_columns = [column_name for column_name in books_df.select_dtypes(include=['Float64']).columns]
number_attrs_df = books_df[numerical_columns].copy()
number_attrs_df.head()

In [ ]:
# Calculamos el vector normalizado de las variables numéricas que se utilizará para calcular 
# la matriz de similitud.
from sklearn.preprocessing import StandardScaler

numerical_scaler = StandardScaler()
numerical_vectors = numerical_scaler.fit_transform(number_attrs_df)
tensor_numerical_df = pd.DataFrame(
    numerical_vectors,
    index=number_attrs_df.index,
    columns=number_attrs_df.columns
)
tensor_numerical_df.head()

In [ ]:
# Combinamos las dos matrices de características vectorizadas para construir la matriz de similitud.
similarity_tensors_df = tensor_numerical_df.merge(tensor_categorical_df, left_index=True, right_index=True)
similarity_tensors_df.head()

In [ ]:
# Grabamos el dataset de los tensores para calcular la matriz de similaridad
similarity_tensors_df.to_parquet("./dataset/similarity_tensors.parquet", compression="gzip")

In [ ]:
# Finalmente, calculamos la matriz de similitud utilizando el ángulo del coseno entre tensores:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(similarity_tensors_df, similarity_tensors_df)
similarity_matrix_df = pd.DataFrame(similarity_matrix, index=similarity_tensors_df.index, columns=similarity_tensors_df.index)
similarity_matrix_df.head()

In [ ]:
# Grabamos el dataset de la matriz de similaridad
similarity_matrix_df.to_csv("./dataset/similarity_matrix.csv", compression="gzip")

In [ ]:
# Definimos las funciones para el cálculo de recomendaciones:

def get_book_recommendations_by_id(
        books_df: pd.DataFrame,
        similarity_matrix_df: pd.DataFrame, 
        book_id: int,
        recommendations: int
) -> pd.DataFrame:
    if book_id not in books_df.index.tolist():
        return []
    book_similarities = list(enumerate(similarity_matrix_df[book_id]))
    book_similarities = sorted(book_similarities, key=lambda x: x[1], reverse=True)
    most_similar_books = book_similarities[1:1+recommendations]
    book_indices = [i[0] for i in most_similar_books] 
    return books_df[['original_title', 'author']].iloc[book_indices]

def get_book_recommendations_by_title(
        books_df: pd.DataFrame, 
        similarity_matrix_df: pd.DataFrame, 
        book_title: str,
        recommendations: int
) -> pd.DataFrame:
    book_id = get_book_id_by_title(books_df, book_title)
    return get_book_recommendations_by_id(books_df, similarity_matrix_df, book_id, recommendations)

In [ ]:
# Stefan Zweig: Beware of Pity
# Oscar Wilde: The picture of Dorian Gray
# George Orwell: 1984

get_book_recommendations_by_title(books_df, similarity_matrix_df, "Beware of Pity", 10)
books_df[books_df['book_title'].str.contains('1984')]

In [ ]:
get_book_recommendations_by_title(books_df, similarity_matrix_df, "The picture of Dorian Gray", 10)

In [ ]:
get_book_recommendations_by_title(books_df, similarity_matrix_df, "1984", 10)

### Pregunta 1:

Una editorial nos ha contactado para ver qué parámetros debería tener un libro para que fuera **exitoso**. A partir del dataset y su análisis, orienta a la editorial sobre qué parámetros deben seguir a la hora de publicar un nuevo libro.

**Enfoque de Solución**
Listamos los libros más exitosos como: 
* los que tienen positive_ratings >= mediana, 
* los que tienen premios (awards), 
* los que tienen average_rating >= media.

A partir de ese conjunto de datos, buscamos las características comunes en:
* Géneros
* Formato
* Autor
* Idioma
* Promedio de Num. de Páginas

In [ ]:
# Búsqueda de datos a través de filtros
positives_df = books_df[books_df['positive_ratings'] >= books_df['positive_ratings'].median()]
awarded_df = books_df[books_df['awards'].notna() & books_df['awards'] != ""]
ratings_df = books_df[books_df['average_rating'] > 3.5]

# Unión de resultados
successful_books_df = pd.concat([positives_df, awarded_df, ratings_df], ignore_index=True, verify_integrity=True)
successful_books_df.sort_values(by=['awards','positive_ratings','average_rating'], axis='rows', ascending=False, inplace=True)

# Encontramos las modas
top_100_df = successful_books_df[['genres','format','author','book_language','num_pages']].head(100)

In [ ]:
from collections import Counter
top_genres = top_100_df['genres'].tolist()
top_genres = list(set(top_genres))
top_genres = " ".join(top_genres).split(" ")
Counter(top_genres).most_common(10)

In [ ]:
top_formats = top_100_df['format'].tolist()
Counter(top_formats).most_common(3)

In [ ]:
top_authors = top_100_df['author'].tolist()
Counter(top_authors).most_common(5)

In [ ]:
top_languages = top_100_df['book_language'].tolist()
Counter(top_languages).most_common(3)

In [ ]:
mean_numpages = top_100_df['num_pages'].median()
devstd_numpages = top_100_df['num_pages'].std()
[devstd_numpages, mean_numpages] 

### Pregunta 2:
Obtenemos las recomendaciones de los libros:

   * **"A Court of Thorns and Roses" de Sarah J. Maas**: NO está en la lista
   * **"La Apología de Sócrates" de Platón**: NO está en la lista
   * **"Hamlet" de William Shakespeare**: SI está en la lista

In [ ]:
{
    "A Court of Thorns and Roses": get_book_recommendations_by_title(books_df, similarity_matrix_df, "A Court of Thorns and Roses", 10),
    "La Apologia de Socrates": get_book_recommendations_by_title(books_df, similarity_matrix_df, "La Apologia de Socrates", 10),
    "La Apología de Sócrates": get_book_recommendations_by_title(books_df, similarity_matrix_df, "La Apología de Sócrates", 10),
    "Hamlet": get_book_recommendations_by_title(books_df, similarity_matrix_df, "Hamlet", 10)
}

In [ ]:
books_df[books_df['author']=='plato']

In [ ]:
books_df[books_df['book_title'].str.contains('socrates')]